In [1]:
%matplotlib inline

from matplotlib import pyplot as plt
import numpy as np

from music21.note import Note
from music21.pitch import Pitch
from music21.interval import Interval, DiatonicInterval
from music21.stream import Stream
from music21.lily.translate import LilypondConverter
from music21 import scale
from music21 import clef
from music21 import environment

us = environment.UserSettings()
us['musicxmlPath'] = '/usr/bin/musescore'
us['musescoreDirectPNGPath'] = '/usr/bin/musescore'
us['musicxmlPath']

from itertools import product
from collections import Counter

## TODO:

- can apply is_good criterea in a specific order while number of results is > 0

In [2]:
perfect_fifths = [
    DiatonicInterval('perfect', 5),
    DiatonicInterval('perfect', 12),
]

perfect_octaves = [
    DiatonicInterval('perfect', 1),
    DiatonicInterval('perfect', 8),
    DiatonicInterval('perfect', 15),
]

perfect_intervals = perfect_fifths + perfect_octaves

cantizans = [
    DiatonicInterval('major', 7),
    DiatonicInterval('major', 14),
]

tenorizans = [
    DiatonicInterval('major', 2),
    DiatonicInterval('major', 9),
    DiatonicInterval('major', 16),
]

vertical_intervals = [
    DiatonicInterval('major', 3),
    DiatonicInterval('minor', 3),
    DiatonicInterval('perfect', 5),
    DiatonicInterval('major', 6),
    DiatonicInterval('minor', 6),
    DiatonicInterval('perfect', 8),
    DiatonicInterval('major', 10),
    DiatonicInterval('minor', 10),
]

optional_verticals = [
    DiatonicInterval('perfect', 12),
    DiatonicInterval('major', 13),
    DiatonicInterval('minor', 13),
    DiatonicInterval('perfect', 15),
    DiatonicInterval('major', 17),
    DiatonicInterval('minor', 17),
]

melodic_intervals = [
    'm2', 'M2', 'm3', 'M3', 'P4', 'P5', 'm6'
]

arpeggio_intervals = [
    ['M3', 'm3'],
    ['m3', 'M3'],
    ['M3', 'P4'],
    ['m3', 'P4'],
    ['P4', 'm3'],
    ['P4', 'M3'],
    ['M-3', 'm-3'],
    ['m-3', 'M-3'],
    ['M-3', 'P-4'],
    ['m-3', 'P-4'],
    ['P-4', 'm-3'],
    ['P-4', 'M-3'],
]

In [11]:
def get_harmonies(register, cantus_firmus, is_wide=True):
    # 1-note filters
    # Allowed vertical intervals
    if is_wide:
        allowed_intervals = vertical_intervals + optional_verticals
    else:
        allowed_intervals = vertical_intervals
        
    if Interval(cantus_firmus[-2], cantus_firmus[-1]).direction == 1:
        penultimate_intervals = tenorizans
    else:
        penultimate_intervals = cantizans
    first = [n for n in register if Interval(cantus_firmus[0], n) in perfect_intervals]
    penultimate = [n for n in register if Interval(cantus_firmus[-1], n) in penultimate_intervals]
    last = [n for n in register if Interval(cantus_firmus[-1], n) in perfect_octaves]
    others = [[n for n in register if Interval(cf, n) in allowed_intervals] for cf in cantus_firmus[1:-2]]
    singles = [first] + others + [penultimate] + [last]
    
    return singles

def is_melodic(pair):
    return Interval(*pair).name in melodic_intervals

def is_repeating(pair):
    return pair[0] == pair[1]

def is_parallel_5_or_8(pair, cf):
    interval1 = Interval(cf[0], pair[0])
    interval2 = Interval(cf[1], pair[1])
    if (interval1 in perfect_fifths) and (interval2 in perfect_fifths):
        return True
    if (interval1 in perfect_octaves) and (interval2 in perfect_octaves):
        return True
    return False

def is_oposite_motion_to_perfect(pair, cf):
    h_interval2 = Interval(cf[1], pair[1])
    m_interval1 = Interval(cf[0], cf[1])
    m_interval2 = Interval(pair[0], pair[1])
    
    if h_interval2 in perfect_intervals:
        return m_interval1.direction != m_interval2.direction
    return True

def is_exchanging_voices(pair, cf):
    # TODO: maybe with `simplename` instead of `name`
    if pair[0].name == cf[1].name and pair[1].name == cf[0].name:
        return True
    return False

def is_valid_pair(pair, cantus_firmus):
    return (
        is_melodic(pair) 
        and not is_repeating(pair)
        and not is_parallel_5_or_8(pair, cantus_firmus)
        and is_oposite_motion_to_perfect(pair, cantus_firmus)
        and not is_exchanging_voices(pair, cantus_firmus)
    )

def get_pairs(harmonies, cantus_firmus, predicate):
    pairs = []
    for idx, (left, right) in enumerate(zip(harmonies[:-1], harmonies[1:])):
        steps = []
        for pair in product(left, right):
            if predicate(pair, cantus_firmus[idx:idx+2]):
                steps.append(pair)
        pairs.append(steps)
    
    return pairs

In [4]:
def is_continuation(tuplet_pair):
    first, second = tuplet_pair
    return first[1:] == second[:-1]

def is_arpeggio(triplet):
    i1 = Interval(triplet[0], triplet[1]).directedSimpleName
    i2 = Interval(triplet[1], triplet[2]).directedSimpleName
    
    return [i1, i2] in arpeggio_intervals

def is_repeating_intervals(triplet, cf):
    i1 = Interval(triplet[0], cf[0]).simpleName
    i2 = Interval(triplet[1], cf[1]).simpleName
    i3 = Interval(triplet[2], cf[2]).simpleName
    
    return i1[-1] == i2[-1] == i3[-1]

def is_closing_leap(triplet):
    i1 = Interval(triplet[0], triplet[1]).chromatic.semitones
    i2 = Interval(triplet[1], triplet[2]).chromatic.semitones
    
    sign = i1 // abs(i1)
    if abs(i1) >= 5 and (i2 not in [-sign * 1, -sign * 2]):
        return False
    return True

def is_valid_triplet(triplet, cantus_firmus):
    return (
        not is_arpeggio(triplet)
        and not is_repeating_intervals(triplet, cantus_firmus)
        and is_closing_leap(triplet)
    )

def generate_next_tuplet(previous_tuplet, cantus_firmus, predicate):
    next_tuplets = []
    for idx, (left, right) in enumerate(zip(previous_tuplet[:-1], previous_tuplet[1:])):
        steps = []
        for pair in product(left, right):
            if is_continuation(pair):
                next_tuplet = pair[0] + pair[1][-1:]
                if predicate(next_tuplet, cantus_firmus[idx:idx+len(next_tuplet)]):
                    steps.append(list(next_tuplet))
        next_tuplets.append(steps)
    
    return next_tuplets

def is_jumpy(quadruplet):
    return all([abs(Interval(p1, p2).chromatic.semitones) > 3 for p1, p2 in zip(quadruplet[:-1], quadruplet[1:])])

def is_valid_quadruplet(quadruplet, cf):
    return not is_jumpy(quadruplet)

In [5]:
def preffered_vertical_intervals(tuplet, cf, more, less):
    good = []
    bad = []
    for t, c in zip(tuplet, cf):
        intv = int(Interval(c, t).simpleName[-1])
        if intv in more:
            good.append(intv)
        elif intv in less:
            bad.append(intv)
    
    return len(good) > len(bad)

def has_more_imperfect_intervals(tuplet, cf):
    return preffered_vertical_intervals(tuplet, cf, (3, 6), (1, 5, 8))
    
def has_more_5ths_than_8ths(tuplet, cf):
    return preffered_vertical_intervals(tuplet, cf, (5,), (1, 8))

def has_more_steps_than_leaps(tuplet):
    steps = []
    leaps = []
    for first, second in zip(tuplet[:-1], tuplet[1:]):
        intv = int(Interval(first, second).name[-1])
        if intv <= 2:
            steps.append(intv)
        else:
            leaps.append(intv)
    return len(steps) > len(leaps)

def has_more_contrary_motion_than_not(tuplet, cf):
    contrary = []
    other = []
    for p1, p2, c1, c2 in zip(tuplet[:-1], tuplet[1:], cf[:-1], cf[1:]):
        ip = Interval(p1, p2)
        ic = Interval(c1, c2)
        
        if ip.direction != ic.direction:
            contrary.append(1)
        else:
            other.append(1)
    return len(contrary) > len(other)

def is_varied(tuplet):
    counts = Counter(tuplet)
    if all(val <= 3 for val in counts.values()) and (list(counts.values()).count(3) < 2):
        for idx in range(len(tuplet)-4):
            if tuplet[idx] == tuplet[idx+2] == tuplet[idx+4]:
                return False
        for idx in range(len(tuplet)-3):
            if (tuplet[idx] == tuplet[idx+2]) and (tuplet[idx+1] == tuplet[idx+3]):
                return False
        return True
    return False

def is_good(tuplets, cf):
    return (
        has_more_imperfect_intervals(tuplets, cf)
        and has_more_steps_than_leaps(tuplets)
        and is_varied(tuplets)
        and has_more_contrary_motion_than_not(tuplets, cf)
    )

In [6]:
def do_it(cantus_firmus, register, is_wide=True):
    harmonies = get_harmonies(register, cantus_firmus, is_wide=is_wide)
    pairs = get_pairs(harmonies, cantus_firmus, is_valid_pair)
    triplets = generate_next_tuplet(pairs, cantus_firmus, is_valid_triplet)
    quadruplets = generate_next_tuplet(triplets, cantus_firmus, is_valid_quadruplet)
    results = quadruplets
    while len(results) > 1:
        results = generate_next_tuplet(results, cantus_firmus, lambda x, y: True)
    results = [r for r in results[0] if is_good(r, cantus_firmus)]
    
    return results

def lily(results, cantus_firmus):
    conv = LilypondConverter()
    print(' '.join([str(conv.lyPitchFromPitch(p))[:-1] + '2' for p in cantus_firmus]))
    print()

    for r in results:
        print(' '.join([str(conv.lyPitchFromPitch(p))[:-1] + '2' for p in r]))

In [7]:
tone = scale.MajorScale('c')

In [8]:
soprano = tone.getPitches('c4', 'a5')
alto = tone.getPitches('f3', 'd5')
tenor = tone.getPitches('c3', 'a4')
bass = tone.getPitches('f2', 'd4')

In [9]:
cf1 = [Pitch(p) for p in ('c4', 'a3', 'g3', 'e3', 'f3', 'a3', 'g3', 'e3', 'd3', 'c3')]
assert all(c in tenor for c in cf1)
cf2 = [Pitch(p) for p in ('c3', 'd3', 'f3', 'e3', 'd3', 'c3', 'a2', 'b2', 'c3')]
assert all(c in bass for c in cf2)
cf3 = [Pitch(p) for p in ('c4', 'e4', 'd4', 'g4', 'a4', 'g4', 'e4', 'f4', 'd4', 'c4')]
assert all(c in alto for c in cf3)

In [12]:
results = do_it(cf1, soprano, is_wide=True)
lily(results, cf1)

c'2 a2 g2 e2 f2 a2 g2 e2 d2 c2

g'2 a'2 b'2 c''2 d''2 a'2 b'2 g'2 b'2 c''2
g''2 f''2 g''2 c''2 d''2 a'2 b'2 g'2 b'2 c''2
